# SDPG(Corbel2301_block2_June2019_crop_ali_crop)

In [ ]:
import logging
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
#from ipywidgets import *
import cv2
import time
#import kernels
import skimage
#from skimage import io as skimage_io
import mrcfile
import logging
import information_theory
from collections import namedtuple

In [ ]:
from my_google_auth import DriveHandler
service = DriveHandler.get_drive_service()
handler = DriveHandler.DriveHandler(service)
MY_SHARED_DRIVE_ID = '1hGHvkP46fxLCQbUlyYhAS_eVl6PollQM'

In [ ]:
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
vol_filename = "Corbel2301_block2_June2019_crop_ali_crop.mrc"

In [ ]:
Args = namedtuple("args", ["input", "output"])
fn, fe = vol_filename.split(".")
args = Args(vol_filename , fn + "_denoised." + fe)
print(args)

In [ ]:
%%bash
ln -sf /nas/vruiz/vols/Corbel2301_block2_June2019_crop_ali_crop.mrc .

In [ ]:
#noisy = skimage.io.imread(args.input, plugin="tifffile").astype(np.float32)
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data

In [ ]:
Z_dim = noisy.shape[0]
Z2 = Z_dim//2

In [ ]:
plt.imshow(noisy[Z2], cmap="gray")

In [ ]:
def get_gaussian_kernel(sigma=1):
    number_of_coeffs = 3
    number_of_zeros = 0
    while number_of_zeros < 2 :
        delta = np.zeros(number_of_coeffs)
        delta[delta.size//2] = 1
        coeffs = scipy.ndimage.gaussian_filter1d(delta, sigma=sigma)
        number_of_zeros = coeffs.size - np.count_nonzero(coeffs)
        number_of_coeffs += 1
    return coeffs[1:-1]

sigma = np.array([1.5, 1.5, 1.5])
kernel = [None]*3
kernel[0] = get_gaussian_kernel(sigma[0])
kernel[1] = get_gaussian_kernel(sigma[1])
kernel[2] = get_gaussian_kernel(sigma[2])

In [ ]:
import denoising.volume.OF_gaussian as SPGD

In [ ]:
denoiser = SPGD.Monochrome_Denoising(logger)

In [ ]:
denoised = denoiser.filter(noisy, kernel, l=1, w=5)

In [ ]:
with mrcfile.new(f"denoised.mrc", overwrite=True) as mrc:
    mrc.set_data(denoised.astype(np.float32))
    mrc.data

In [ ]:
def read_MRC(file_path):
    return mrcfile.read(file_path)

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='denoised.mrc',
    drive_file_name=f'{vol_filename}__denoised__SPGD.mrc',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[25], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[25], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[16], denoised[16])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[1][300:,300:], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[1][300:,300:], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[16][300:,300:], denoised[16][300:,300:])}")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[:, 100], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, 100], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, 100], denoised[:, 100])})")
fig.tight_layout()
plt.show()

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy[7][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("FlowDenoising")
plt.imshow(denoised[7][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[7][::-1, :] - denoised[7][::-1, :], cmap="gray")

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(16, 16))
slice_idx = denoised.shape[0]//2
plt.imshow(denoised[slice_idx, 200:600, 200:600], cmap="gray")
plt.savefig("denoised.pdf", bbox_inches='tight')

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='denoised.pdf',
    drive_file_name=f'{vol_filename}__denoised__SPGD.pdf',
    drive_folder_id=MY_SHARED_DRIVE_ID
)